In [ ]:
import pyFAI
import fabio
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os, re
from maskGeneratorBM31 import makeDataSet, makeMasks, integrateAverage, integrateIndividual


In [ ]:
%matplotlib inline

direc = r'C:\Users\kenneth1a\Documents\beamlineData\moduleDistortion\LaB6\xrd/' # Directory of xrd files
os.chdir(direc)

dest = direc.replace(r'X:\staff\july2023',r'C:\Users\kenneth1a\Documents\beamlineData\July2023')

if not os.path.exists(dest):
    os.makedirs(dest)

mask  = r'Z:\bm31\inhouse\july2023/pdf_baseMask_tilt.edf' # Mask file
mask = fabio.open(mask).data
poni  = r'Z:\bm31\inhouse\july2023/Si_15tilt_0p25579A.poni' # Poni file
poni = pyFAI.load(poni)
wavelength = poni.wavelength*10**10
gainFile = r'C:\Users\kenneth1a\Documents\beamlineData\July2023\gainmap\calculatedGainMap_48p6keV_filtered_kpm_2023-07-21.edf'


badFramesLog = f'{dest}/badFrames.txt'
if os.path.isfile(badFramesLog):
    os.remove(badFramesLog)

files = glob('*.cbf')
files.sort()
i1 = fabio.open(files[0]).data
dataset = np.empty(shape = (*i1.shape,len(files)))

doMonitor = True

#monitorfile = glob('*.dat')[0]
#monitor = np.loadtxt(monitorfile,usecols = 2, skiprows = 1)

scale = 10**9
dataset, usedFiles = makeDataSet(files, badFramesLog, scale, doMonitor)

average = np.average(dataset,axis=2)
median = np.median(dataset,axis=2)
vmax = np.percentile(np.where(np.isnan(average),0,average),99.9)
plt.figure()
plt.imshow(average,vmax = vmax)
plt.colorbar()
plt.show()

In [ ]:
plt.figure()
stdev = np.std(dataset,axis = 2)
vmax = np.percentile(np.where(np.isnan(stdev),0,stdev),99.9)
plt.imshow(np.std(dataset,axis = 2),vmax = vmax)
plt.colorbar()

In [ ]:
%matplotlib nbagg

nstdevs = 3
maskdct = makeMasks(dataset,usedFiles,mask, nstdevs = nstdevs,plot = True)

In [ ]:
subdir = f'xye_{nstdevs}stdevs'


integrateAverage(dataset, usedFiles, poni, gainFile, maskdct,dest)

integrateIndividual(dataset = dataset,files = usedFiles, dest =  dest, subdir = subdir,poni= poni, maskdct=maskdct, 
                    gainFile = gainFile)
